In [ ]:
from edesdetectrl.environments.binary_classification import _get_dist_reward_impl


def r1(x, i, gt):
    return 1 if x == gt[i] else 0


def r2(x, i, gt):
    return 1 if x == gt[i] else -1


def r3(x, i, gt):
    return _get_dist_reward_impl(x, i, gt)


def r4(x, i, gt):
    if x == gt[i]:
        # Find the frame difference between the current frame and the first ground truth that
        # matches the prediction to the left of the frame.
        closest_left = 0
        while gt[i - closest_left] == x:
            closest_left += 1
            if i - closest_left < 0:
                closest_left = None
                break

        if closest_left is not None:
            return float(10 - closest_left)
        else:
            return 10
    else:
        return -1.0


# Hello, World


In [ ]:
seq = [0,0,0,0,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,1,1,1,1,1,1]


In [ ]:
import numpy as np


def get_true_q(seq, r, discount):
    q = []
    seq_len = len(seq)
    for t in range(seq_len):
        r_0 = r(0, t, seq)
        r_1 = r(1, t, seq)

        future_returns = np.sum(
            [r(seq[k], k, seq) * discount ** (k - t) for k in range(t + 1, seq_len)]
        )

        q.append([r_0 + future_returns, r_1 + future_returns])

    return np.array(q)


import matplotlib.pyplot as plt


def add_gt_to_plot(ax):
    for i in range(len(seq)):
        ax.axvspan(i - 0.5, i + 0.5, facecolor="g" if seq[i] == 0 else "b", alpha=0.1)


discount = 0.9
seq_len = len(seq)
q_r1_full = get_true_q(seq * 100, r1, discount)
q_r2_full = get_true_q(seq * 100, r2, discount)
q_r3_full = get_true_q(seq * 100, r3, discount)
q_r4_full = get_true_q(seq * 100, r4, discount)


In [ ]:
q_r1 = q_r1_full[seq_len : 2 * seq_len]
q_r2 = q_r2_full[seq_len : 2 * seq_len]
q_r3 = q_r3_full[seq_len : 2 * seq_len]
q_r4 = q_r4_full[seq_len : 2 * seq_len]

fig, ax = plt.subplots(ncols=4)
ax[0].plot(q_r1[:, 0])
ax[0].plot(q_r1[:, 1])
add_gt_to_plot(ax[0])
ax[1].plot(q_r2[:, 0])
ax[1].plot(q_r2[:, 1])
add_gt_to_plot(ax[1])
ax[2].plot(q_r3[:, 0])
ax[2].plot(q_r3[:, 1])
add_gt_to_plot(ax[2])
ax[3].plot(q_r4[:, 0])
ax[3].plot(q_r4[:, 1])
add_gt_to_plot(ax[3])
fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(ncols=4)
ax[0].plot(q_r1[:, 0] - q_r1[:, 1])
add_gt_to_plot(ax[0])
ax[1].plot(q_r2[:, 0] - q_r2[:, 1])
add_gt_to_plot(ax[1])
ax[2].plot(q_r3[:, 0] - q_r3[:, 1])
add_gt_to_plot(ax[2])
ax[3].plot(q_r4[:, 0] - q_r4[:, 1])
add_gt_to_plot(ax[3])
fig.tight_layout()
